# Location context

This notebook calculates pixel values for regions of interest (ROI) in each video, then creates a dataframe with True or False for whether the animal was in each ROI (using SLEAP coordinates) at each frame of the video.

In [1]:
# Packages
import cv2
import h5py
import numpy as np
import pandas as pd
import glob
import os
import re
from scipy.interpolate import interp1d

Run the code with manual changes to file names for EACH video. 
Some videos differ slightly in pixel coordinates of ROIs so we do not set one ROI loaction for all videos, however the location of these 
does not change within the course of the video.

In [5]:
# Load video
video_path = r"C:\Users\irs3th\keypoint-moseq\home_cage\saline_orfo_arena2_2785_re_15s.mp4" # insert your path to the video (mp4)
cap = cv2.VideoCapture(video_path)

# Read any example frame 
frame_number = 100
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
ret, frame = cap.read()

if ret:
    # Save as an image
    cv2.imwrite('extracted_frame.jpg', frame)
else:
    print("Error extracting frame")

cap.release()

In [6]:
# Load the SLEAP H5 file
h5_file_path = r"C:\Users\irs3th\keypoint-moseq\home_cage\saline_orfo_arena2_2785_re_15s.h5"
with h5py.File(h5_file_path, "r") as f:
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

# Fill in missing values (code from https://sleap.ai/notebooks/Analysis_examples.html) 
def fill_missing(locations):
    for i in range(locations.shape[1]):  # Loop through each body part
        for j in range(locations.shape[2]):  # Loop through each coordinate (x or y)
            mask = np.isnan(locations[:, i, j, 0])
            idx = np.where(~mask, np.arange(mask.shape[0]), 0)
            idx = np.maximum.accumulate(idx, axis=0)
            locations[:, i, j, 0] = locations[idx, i, j, 0]
    return locations

# Save as locations
locations = fill_missing(locations)

## Shelter location

In [7]:
# Draw a box around region of interest (shelter) to get the coordinate location

# Press 'r' to reset the cropping region
# Press 'c' or 'q' to quit the program and close the window

refPt = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        # draw a rectangle around the ROI
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", image)

# Load the image
image = cv2.imread('extracted_frame.jpg')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# Keep looping until the 'q' or 'c' key is pressed
while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        image = clone.copy()

    elif key == ord("c") or key == ord("q"):
        break

# Close open windows
cv2.destroyAllWindows()

# Print the coordinates
if len(refPt) == 2:
    print(f'shelter coordinates: {refPt}')
    x1, y1 = refPt[0]
    x2, y2 = refPt[1]
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    print(f'shelter width: {width} pixels')
    print(f'shelter height: {height} pixels')

shelter coordinates: [(552, 30), (738, 206)]
shelter width: 186 pixels
shelter height: 176 pixels


In [8]:
# Create a dataframe with True (in shelter) or False (not in shelter) for every frame in video

# Define the coordinates from above (top-left and bottom-right)
x1, y1 = 552, 30
x2, y2 = 738, 206

# Index for center 3 location (middle of the mouse)
CENTER3_INDEX = 5
center3_loc = locations[:, CENTER3_INDEX, :, :]

# Create a list to store whether the animal is in the shelter for each frame
in_box = []

for frame in range(center3_loc.shape[0]):
    x, y = center3_loc[frame, 0, 0], center3_loc[frame, 1, 0]
    if x1 <= x <= x2 and y1 <= y <= y2:
        in_box.append(True)
    else:
        in_box.append(False)

# Create a DF with the frame index and whether the animal is in the shelter
df = pd.DataFrame({
    'frame_index': range(center3_loc.shape[0]),
    'shelter': in_box
})

df.insert(0, 'id', '2785') # manually change
df.insert(1, 'group', 'saline_orfo') # manually change

# Save to a CSV file
df.to_csv("shelter_saline_orfo_2785.csv", index=False) # manually change

# Check for total time spent in the shelter (can save this info if you would like)
frames_in_box = sum(in_box)
fps = 25
time_in_box_seconds = frames_in_box / fps

print(f'Total number of frames spent in the shelter: {frames_in_box}')
print(f'Total time spent in the shelter: {time_in_box_seconds:.2f} seconds')

Total number of frames spent in the shelter: 215
Total time spent in the shelter: 8.60 seconds


## Food hopper location

In [9]:
# Draw a box around region of interest (hopper) to get the coordinate location
# I make the box very small to just get where the nose goes in the hopper and small portion of the cage floor, but define how you want

# Press 'r' to reset the cropping region.
# Press 'c' or 'q' to quit the program and close the window.

refPt = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        # draw a rectangle around the ROI
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", image)

# Load the image
image = cv2.imread('extracted_frame.jpg')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# Keep looping until the 'q' or 'c' key is pressed
while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        image = clone.copy()

    elif key == ord("c") or key == ord("q"):
        break

# Close open windows
cv2.destroyAllWindows()

# Print the coordinates
if len(refPt) == 2:
    print(f'food hopper coordinates: {refPt}')
    x1, y1 = refPt[0]
    x2, y2 = refPt[1]
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    print(f'hopper width: {width} pixels')
    print(f'hopper height: {height} pixels')

food hopper coordinates: [(681, 282), (778, 442)]
hopper width: 97 pixels
hopper height: 160 pixels


In [10]:
# Create a dataframe with True (by food hopper) or False (not by food hopper) for every frame in video

# Define the coordinates from above (top-left and bottom-right)
x1, y1 = 681, 282
x2, y2 = 778, 442

# Index for nose (since I am interested in sniffing near the hopper)
NOSE_INDEX = 0
nose_loc = locations[:, NOSE_INDEX, :, :]

# Create a list to store whether the animal is near the hopper for each frame
in_box = []

for frame in range(nose_loc.shape[0]):
    x, y = nose_loc[frame, 0, 0], nose_loc[frame, 1, 0]
    if x1 <= x <= x2 and y1 <= y <= y2:
        in_box.append(True)
    else:
        in_box.append(False)


# DF with the frame index and whether the animal is near the hopper
df = pd.DataFrame({
    'frame_index': range(nose_loc.shape[0]),
    'food_hopper': in_box
})

df.insert(0, 'id', '2785') # manually change
df.insert(1, 'group', 'saline_orfo') # manually change

# Save to a CSV file
df.to_csv("food_hopper_saline_orfo_2785.csv", index=False) # manually change

# Additional calculations for time nose is within the region
frames_in_box = sum(in_box)
fps = 25
time_in_box_seconds = frames_in_box / fps

print(f'Total number of frames spent by the hopper: {frames_in_box}')
print(f'Total time spent by the hopper: {time_in_box_seconds:.2f} seconds')

Total number of frames spent by the hopper: 31
Total time spent by the hopper: 1.24 seconds


## Combine shelter and hopper csv if you want both

In [11]:
# read csv files
shelter = pd.read_csv('shelter_saline_orfo_2785.csv')
food = pd.read_csv('food_hopper_saline_orfo_2785.csv')

# Merge dataframes on the 'id,' 'group,' and 'frame_index' columns
output1 = pd.merge(shelter, food, on=['id', 'group', 'frame_index'], how='outer')

# Save to a CSV 
output1.to_csv("location_saline_orfo_2785.csv", index=False)

## If you want, define region by the water spout to define approaches to drink

In [12]:
# Draw a box around region of interest (water bottle spout) to get the coordinate location
# I draw a small box around the tip of the spout

# Press 'r' to reset the cropping region.
# Press 'c' or 'q' to quit the program and close the window.

refPt = []
cropping = False

def click_and_crop(event, x, y, flags, param):
    global refPt, cropping

    if event == cv2.EVENT_LBUTTONDOWN:
        refPt = [(x, y)]
        cropping = True

    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x, y))
        cropping = False

        # draw a rectangle around the region of interest
        cv2.rectangle(image, refPt[0], refPt[1], (0, 255, 0), 2)
        cv2.imshow("image", image)

# Load the image
image = cv2.imread('extracted_frame.jpg')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

# Keep looping until the 'q' or 'c' key is pressed
while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("r"):
        image = clone.copy()

    elif key == ord("c") or key == ord("q"):
        break

# Close open windows
cv2.destroyAllWindows()

# Print the coordinates
if len(refPt) == 2:
    print(f'water coordinates: {refPt}')
    x1, y1 = refPt[0]
    x2, y2 = refPt[1]
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    print(f'water width: {width} pixels')
    print(f'water height: {height} pixels')

water coordinates: [(323, 37), (361, 68)]
water width: 38 pixels
water height: 31 pixels


In [13]:
# Create a dataframe with True (by water) or False (not by water) for every frame in video

# Define the coordinates from above (top-left and bottom-right)
x1, y1 = 323, 37
x2, y2 = 361, 68


# Index for nose (since I am interested in sniffing/drinking near spout)
NOSE_INDEX = 0
nose_loc = locations[:, NOSE_INDEX, :, :]

# Create a list to store whether the animal is near for each frame
in_box = []

for frame in range(nose_loc.shape[0]):
    x, y = nose_loc[frame, 0, 0], nose_loc[frame, 1, 0]
    if x1 <= x <= x2 and y1 <= y <= y2:
        in_box.append(True)
    else:
        in_box.append(False)

# Create a DF with the frame index and whether the animal is by spout
df = pd.DataFrame({
    'frame_index': range(nose_loc.shape[0]),
    'water': in_box
})

df.insert(0, 'id', '2785') # manually change
df.insert(1, 'group', 'saline_orfo') # manually change

# Save to a csv file
df.to_csv("water_saline_orfo_2785.csv", index=False) # manually change

# Additional calculations for total time spent by spout
frames_in_box = sum(in_box)
fps = 25
time_in_box_seconds = frames_in_box / fps

print(f'Total number of frames spent by the water: {frames_in_box}')
print(f'Total time spent by the water: {time_in_box_seconds:.2f} seconds')


Total number of frames spent by the water: 0
Total time spent by the water: 0.00 seconds


## Merge the shelter/hopper csv with water csv if you want all 3

In [14]:
# Set the working directory 
os.chdir(r"C:\Users\irs3th\keypoint-moseq\home_cage\results")

# CSV files
water = pd.read_csv('water_saline_orfo_2785.csv')
main = pd.read_csv('location_saline_orfo_2785.csv')

# Merge the DFs on the 'id,' 'group,' and 'frame_index' columns
output1 = pd.merge(water, main, on=['id', 'group', 'frame_index'], how='outer')

# Save to a CSV 
output1.to_csv("location_saline_orfo_2785_new.csv", index=False) # replace with new df

Move on to location_behavior.ipynb